# Unet模型裁剪labelme生成的图片

In [1]:
import os
import sys
import cv2
import shutil
import numpy as np
from PIL import Image
from imageio import imread,imwrite

sys.path.insert(0, 'trainer')

from mbsh.core.unet.data import *
from mbsh.core.unet_pp.segmentation_models import Unet, Xnet
# from mbsh.core.images import fetch_all_files, split_video

2022-05-06 17:53:31,744 - INFO - __init__.py - init_logger - 58 - ini  logger file D:\projects\znyx-trainer\logs\kernel-2a648292-f594-4fb4-a47b-6c1f65be48a2.log


use temp dir:d:\znyx\temp


Using TensorFlow backend.
2022-05-06 17:53:33,981 - DEBUG - __init__.py - wrapper - 241 - (private) matplotlib data path: C:\ProgramData\Anaconda3\envs\tf1\lib\site-packages\matplotlib\mpl-data
2022-05-06 17:53:33,983 - DEBUG - __init__.py - wrapper - 241 - matplotlib data path: C:\ProgramData\Anaconda3\envs\tf1\lib\site-packages\matplotlib\mpl-data
2022-05-06 17:53:33,988 - DEBUG - __init__.py - wrapper - 241 - CONFIGDIR=C:\Users\Administrator\.matplotlib
2022-05-06 17:53:33,991 - DEBUG - __init__.py - <module> - 1480 - matplotlib version 3.3.4
2022-05-06 17:53:33,992 - DEBUG - __init__.py - <module> - 1481 - interactive is False
2022-05-06 17:53:33,993 - DEBUG - __init__.py - <module> - 1482 - platform is win32
2022-05-06 17:53:33,993 - DEBUG - __init__.py - <module> - 1483 - loaded modules: ['builtins', 'sys', '_frozen_importlib', '_imp', '_warnings', '_thread', '_weakref', '_frozen_importlib_external', '_io', 'marshal', 'nt', 'winreg', 'zipimport', 'encodings', 'codecs', '_codecs',

2022-05-06 17:53:34,044 - DEBUG - __init__.py - wrapper - 241 - CACHEDIR=C:\Users\Administrator\.matplotlib
2022-05-06 17:53:34,048 - DEBUG - font_manager.py - <module> - 1436 - Using fontManager instance from C:\Users\Administrator\.matplotlib\fontlist-v330.json


## 模型配置

In [2]:
# model_path = r'H:\ai_data\img_crop\train1227\cache\X_ca_region_19-12-27_10_16_19-0.996.hdf5'
# model_path = r'H:\ai_data\img_crop\纵隔超声/cache\X_ca_region_21-05-12_10_21_14-0.995.hdf5'
# model_path = r'H:\ai_data\img_crop\train0907-超声/cache\X_ca_region_21-09-07_14_07_17-0.995.hdf5'
model_path = r'D:\共享文件\X_ca_region_21-09-07_14_07_17-0.995.hdf5'

img_size = (512, 512)
input_size = img_size + (3,)
use_unet = False

# 视频图转换后图像大小
# 不改变尺寸则设置为None
img_resize = None #(512, 512)

In [3]:
import tensorflow as tf
tf.test.is_gpu_available() 

True

In [4]:
tf.__version__

'1.15.5'

## 加载模型及参数


In [5]:
# 创建模型并加载
if use_unet:
#     model = unet(input_size=input_size)
    model = Unet(input_shape=input_size, backbone_name='resnet50', encoder_weights='imagenet', decoder_block_type='transpose')
else:
    model = Xnet(input_shape=input_size, backbone_name='resnet50', encoder_weights='imagenet', decoder_block_type='transpose')

model.load_weights(model_path)

2022-05-06 17:55:52,088 - WARNING - deprecation.py - new_func - 506 - From C:\ProgramData\Anaconda3\envs\tf1\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
2022-05-06 17:55:52,452 - WARNING - module_wrapper.py - _tfmw_add_deprecation_warning - 139 - From C:\ProgramData\Anaconda3\envs\tf1\lib\site-packages\keras\backend\tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



## 图片处理函数

In [6]:
# 根据轮廓点列表计算轮廓面积
def areaCal(contour):
    area = 0
    for i in range(len(contour)):
        area += cv2.contourArea(contour[i])
    return area


def takeLength(elem):
    return areaCal([elem])

def resize_image(img, size):
    """
    resize 保持原图的长宽比
    """
    image = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
#     print(image.size,image.shape,type(image))
    iw, ih = image.size
    w, h = size
    scale = min(w / iw, h / ih)
    nw = int(iw * scale)
    nh = int(ih * scale)

    image = image.resize((nw, nh), Image.BICUBIC)
    new_image = Image.new('RGB', size, (0, 0, 0))
    new_image.paste(image, ((w - nw) // 2, (h - nh) // 2))
    img = np.array(new_image)
    return (scale, nw, nh), img[:, :, ::-1]

def process_img(src_img, img_size):
    src_img_bak = src_img.copy()
    src_h = src_img_bak.shape[0]
    src_w = src_img_bak.shape[1]

    new_w, new_h = img_size

    (scale, nw, nh), img = resize_image(src_img, img_size)

    # trans.resize把img从numpy.uint8类型转换为numpy.float64
    img = trans.resize(img, img_size)

    # reshape把img从(512, 512, 3)转换为(1, 512, 512, 3)
    img = np.reshape(img,(1,)+img.shape)

    # 模型识别图片, results为(1, 512, 512, 1)，是灰度图片
    results = model.predict(np.array(img),verbose=1)

    # gray_img是（512，512）的灰度图，元素类型numpy.float32
    gray_img = results[0][:,:,0]
    gray_img[np.where(gray_img <= 0.3)] = 0

    # heatmap转为灰度图
    bw_heatmap = np.uint8(255 * gray_img)
    bw_heatmap[bw_heatmap != 0] = 255
    if cv2.__version__[0] == '4':
        ai_ctrs, _ = cv2.findContours(bw_heatmap.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    else:
        _, ai_ctrs, _ = cv2.findContours(bw_heatmap.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(ai_ctrs) == 0:
        return False,src_img
        
    ai_ctrs.sort(key=takeLength, reverse=True)
    

    x,y,w,h = cv2.boundingRect(ai_ctrs[0])

    # 映射到原图尺寸的坐标系  
    x = x - (new_w - nw) // 2
    y = y - (new_h - nh) // 2
    
    new_x = 0 if x < 0 else x
    new_y = 0 if y < 0 else y
    
    x1 = int(new_x / scale)
    y1 = int(new_y / scale)
    w1 = int(w / scale)
    h1 = int(h / scale)
    
    img_out = src_img_bak[y1:y1+h1, x1:x1+w1, :]
    
    return True,img_out, (x1, y1, w1, h1)
#     return True,img_out[: , : , : : -1], (x1, y1, w1, h1)


def process_path(imgs_path, labels_path, out_imgs_path, out_labels_path, img_size):
    files = os.listdir(imgs_path)
    for file in files:
        ext = file.split('.')[-1]
        if ext == 'csv':
            continue
            
        abs_file = os.path.join(imgs_path, file)
#         src_img = imread(abs_file)
        try:
            src_img = imread(abs_file)
        except:
            print('read error file: ', abs_file)
            return
        
        print(src_img.shape)
        if 1:#src_img.shape[1] / src_img.shape[0] > 1.2 or src_img.shape[0] / src_img.shape[1] > 1.2:
            result, out_img, rect = process_img(src_img, img_size)
        else:
#             continue
            result, out_img, rect = 1,src_img,None #[0,0,-1,-1]
#             result, out_img, rect = 1,src_img[: , : , : : -1],None #[0,0,-1,-1]
        if not result:
            print('detect rect error file: ', abs_file)
            return

        if img_resize:
            _,out_img = resize_image(out_img, img_resize)
            
        # 处理label
        img_fn = os.path.join(labels_path, file[:])   
        if not os.path.exists(img_fn):
            continue
            
        new_fn = os.path.join(out_imgs_path, file)
#         cv2.imencode('.png', out_img[: , : , : : -1])[1].tofile(new_fn)
        cv2.imencode('.png', out_img)[1].tofile(new_fn)
        

        img = np.array(Image.open(img_fn))
        if rect:
            img = img[rect[1]:rect[1]+rect[3], rect[0]:rect[0]+rect[2]]

        if img_resize:
            _,img = resize_image(img, img_resize)
        
        new_fn = os.path.join(out_labels_path, file)
        img = Image.fromarray(img)
        img.save(new_fn)
    
    
def process_files(path, target_dir, img_size):
    img_fn = os.path.join(path,'img.png')
    try:
        src_img = imread(img_fn)
    except:
        print('read error file: ', img_fn)
        return
    
    result, out_img, rect = process_img(src_img, img_size)
    if not result:
        print('detect rect error file: ', img_fn)
        return
    
    if img_resize:
        print(out_img.shape)
        _,out_img = resize_image(out_img, img_resize)
            
#     fpath,fname = os.path.split(file)
    new_fn = os.path.join(target_dir, 'img_crop.png')
    cv2.imencode('.png', out_img[: , : , : : -1])[1].tofile(new_fn)
        
        
    img_fn = os.path.join(path,'label.png')
#     img = Image.open(img_fn)
    
    img = np.array(Image.open(img_fn))
    img = img[rect[1]:rect[1]+rect[3], rect[0]:rect[0]+rect[2]]
#     print(img.shape,'ttt')

    if img_resize:
        _,img = resize_image(img, img_resize)

    img = Image.fromarray(img, mode='P')
    colormap = label_colormap(255)
    img.putpalette((colormap * 255).astype(np.uint8).flatten())

        
    new_fn = os.path.join(target_dir, 'label_crop.png')
    img.save(new_fn)

In [7]:
in_path = r'D:\项目资料\胆结石\胆结石分割模型'
out_path = in_path + '_out12'
os.makedirs(out_path, exist_ok=True)

subs = os.listdir(in_path)
for sub in subs:
    if 'mask' in sub:
        imgs_path = os.path.join(in_path, sub[:-5])
        labels_path = os.path.join(in_path, sub)

        out_imgs_path = os.path.join(out_path, 'images')
        out_labels_path = os.path.join(out_path, 'labels')
        if not os.path.exists(out_imgs_path):
            os.makedirs(out_imgs_path, exist_ok=True)
        if not os.path.exists(out_labels_path):
            os.makedirs(out_labels_path, exist_ok=True)

        process_path(imgs_path, labels_path, out_imgs_path, out_labels_path, img_size)

2022-05-06 17:56:16,128 - WARNING - module_wrapper.py - _tfmw_add_deprecation_warning - 139 - From C:\ProgramData\Anaconda3\envs\tf1\lib\site-packages\keras\backend\tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



(576, 720, 3)
1/1 [==============================] - 919s 919s/step


AttributeError: 'tuple' object has no attribute 'sort'